In [1]:
import pandas as pd
import numpy as np
import data
import networkx as nx
from networkx.algorithms import community
import uuid
import json

The input data are the relationships betweeen viewers and content. In this case the program content has been limited to program series and excluded movies, live news, special events, sports events, etc. 

The data has also been limited to on-demand viewing as opposed to live television viewing. This has been done because we want to focus on active content consumption (as opposed to 'channel surfing') because our larger goals are to model viwer and content interactions that we might see on streaming services.

Start by loading our viewing data. This data has been engineered upstream and is a sample of a much larger dataset. Logically, it is a table of viewer-content engagement. Each row has one content (i.e. program) and one viewer. Because program to viewer relationships are many-to-many both programs and viewers will repeat within the dataset.

_*Note that this dataset is a small sample of the actual dataset used so results here are not as representitive of the actual viewer / content universe_

In [2]:
viewing = pd.read_csv('./input/viewing.csv')

list all columns and show a subset of the data that illustrates what is in it

In [3]:
viewing.columns

Index(['personkey', 'first_respondentweightreportdate',
       'last_respondentweightreportdate', 'intabweight', 'age', 'gender',
       'race', 'person_education', 'person_education_level', 'countysize',
       'county_size_level', 'householdincome', 'languageofhousehold',
       'headofhoushold_education_level', 'householdsize', 'numberofchildren',
       'numberofadults', 'numberofincomes', 'hascat', 'hasdog',
       'hassvodsubscription', 'hasnetflixsubscription', 'hashulusubscription',
       'hasamazonprimesubscription', 'isnewcarprospectlast3years',
       'isnewtruckprospectlast3years', 'hasmac', 'haspc',
       'weekly_viewing_minutes', 'contentsk', 'programname',
       'program_total_engagement', 'engagement'],
      dtype='object')

In [4]:
viewing[['personkey','age','gender','householdincome','contentsk','programname','engagement']].head()

,personkey,age,gender,householdincome,contentsk,programname,engagement
0,181948-1,31,F,63,4932,PITBULLS AND PAROLEES,74
1,6456233-1,61,M,153,5174,AMERICAN IDOL-MON,54
2,6196189-1,68,M,233,5591,MACGYVER,45
3,238154-1,60,M,68,4337,"KIDS ARE ALRIGHT, THE",54
4,8875071-2,53,M,13,4796,INBETWEEN,52


'personkey' and 'contentsk' are keys for viewers and programs respectively. 'engagement' is a refined feature that serves as a relative measure of how interested the viewer is with the program; based on what proportion of the content that the viewer consumes. For exmple, if the viewer watches every episode of Game of Thrones and watches 100% of the duration of those episodes, the engagement would be 100.

We're going to load the data into a network graph, so we need to do some data preparation

In [5]:
# nodes will be programs and viewers
viewing['personkey'] = 'V' + viewing['personkey'].astype(str)
viewing['contentsk'] = 'P' + viewing['contentsk'].astype(str)

In [6]:
# let's build a unique list of programs
programs = viewing[['contentsk','programname','program_total_engagement']].drop_duplicates()

In [7]:
# build the program node attributes. these attributes will be saved within the graph
attribs = [None] * programs.shape[0]
for i in range(0, programs.shape[0]):
    attribs[i]={'type':'program', 'programname':programs.iloc[i]['programname'],'program_total_engagement':programs.iloc[i]['program_total_engagement']}

In [8]:
# now build the actual program node tuples that the graph requires that combine the keys and the attributes
df_nodes = pd.DataFrame({'contentsk':programs['contentsk'],'attribs':attribs})
node_tuples = [tuple(x) for x in df_nodes[['contentsk','attribs']].values]

In [9]:
# Create a new empty graph and add the program nodes
G = nx.Graph()
G.add_nodes_from(node_tuples)

In [10]:
# now let's prepare the viewer nodes
viewer_cols = ['personkey','age', 'gender',
       'race', 'person_education', 'person_education_level', 'countysize',
       'county_size_level', 'householdincome', 'languageofhousehold',
       'headofhoushold_education_level', 'householdsize', 'numberofchildren',
       'numberofadults', 'numberofincomes', 'hascat', 'hasdog']

# viewers
viewers = viewing[viewer_cols].drop_duplicates()
viewers['person_education'] = viewers.race.fillna('')

In [11]:
# build the viewer node attributes
attribs = [None] * viewers.shape[0]
for i in range(0, viewers.shape[0]):
    attribs[i]={'type':'viewer'
    , 'age':viewers.iloc[i]['age']
    , 'householdincome':viewers.iloc[i]['householdincome']
    , 'numberofchildren':viewers.iloc[i]['numberofchildren']
    , 'race':viewers.iloc[i]['race']
    , 'countysize':viewers.iloc[i]['countysize']
    , 'headofhoushold_education_level':viewers.iloc[i]['headofhoushold_education_level']
    , 'person_education_level':viewers.iloc[i]['person_education_level']
    }

In [12]:
# now build the node tuples
df_nodes = pd.DataFrame({'personkey':viewers['personkey'],'attribs':attribs})
node_tuples = [tuple(x) for x in df_nodes[['personkey','attribs']].values]

In [13]:
# add the viewer nodes to the graph
G.add_nodes_from(node_tuples)

In [14]:
# now we need to define the edges, or the connections between the viewer nodes and the program nodes
# we can save the engagement as an attribute of each edge
links = viewing[['personkey','contentsk', 'engagement']].drop_duplicates()

# like nodes, we add to the graph in bulk with tuples that look like this: (node1, node2, {'edge_attribute1': some_value, 'edge_attribute2': some_value})
attribs = [None] * links.shape[0]
for i in range(0, links.shape[0]):
    attribs[i] = {'engagement':links.iloc[i]['engagement']}

df_links = pd.DataFrame({'personkey':links['personkey'], 'contentsk':links['contentsk'], 'attribs':attribs})
link_tuples = [tuple(x) for x in df_links[['personkey','contentsk','attribs']].values]

# add the edges to the graph
G.add_edges_from(link_tuples)

Look at the graph summary statistics

In [15]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 4723
Number of edges: 39016
Average degree:  16.5217


Now that we have build the graph, we want to extract 'communities' from it to generate a rich taxonomy of viewer/content groups

There are various community generation algorithms (see refences). Based on experimentation, we will use the Clauset-Newman-Moore greedy modularity maximization. This algorithm works well because within the on-demand viewing we find that the networks have a relatively high degree of modularity, where there are sparse connections between nodes in different groups. This reflects that fact that generally, viewers will like certain groups of programming and not like other types of programming at all.

https://networkx.github.io/documentation/stable/reference/algorithms/community.html

https://en.wikipedia.org/wiki/Community_structure

Our overall approach will be the following:

1. Extract the initial set of communities.  This will normally result in a small number of large communities and several smaller ones.

2. Recursively treat each community as an isolated graph and contiune to generate sub-comminities within them.  

This will generate a taxonomy of content and viewers similiar to biological taxonomies of species, music genres, etc.

First we have to define some functions that allow for recursion and setting attributes on each generated community 'group'.

In [16]:
# this function pulls the info out of the community and creates a community group object with some statistics about it
def build_community_group(group_list):
    
#    group_list = list(map(list, group_list))[0]
    
    contentsks = list(filter(lambda x: x.startswith('P'), group_list))
    personkeys = list(filter(lambda x: x.startswith('V'), group_list))

    group_viewing = viewing[viewing['contentsk'].isin(contentsks) & viewing['personkey'].isin(personkeys)]
    
    # group_viewing = viewing[viewing['contentsk'].isin(contentsks[0]) & viewing['personkey'].isin(personkeys[0])]
    
    n_contentsks= len(group_viewing['contentsk'].drop_duplicates())
    n_personkeys = len(group_viewing['personkey'].drop_duplicates())
    
    group_viewers = viewing[viewing['personkey'].isin(personkeys)]
    
    pct_female = float(round(group_viewers[group_viewers['gender']=='F'].count()[0] / group_viewers.count()[0], 2))
    pct_male = float(round(group_viewers[group_viewers['gender']=='M'].count()[0] / group_viewers.count()[0],2))
    pct_children = float(round(group_viewers[group_viewers['numberofchildren']>0].count()[0] / group_viewers.count()[0],2))

    # program_engagement = group_viewing.groupby('programname')['engagement'].sum().to_frame().reset_index().sort_values(by=['engagement'], ascending=False)
    
    # label the group by using the the top n programs based on combined ranking of engagement and number of viewers
    program_engagement = group_viewing.groupby(['contentsk','programname'])['engagement'].sum().reset_index()
    program_engagement['respondents'] = group_viewing.groupby(['contentsk','programname'])['personkey'].count().rank(ascending=False).tolist()
    # program_engagement['combined_rank'] = program_engagement.engagement + program_engagement.respondents
    # program_engagement = program_engagement.sort_values(by=['combined_rank'])
    program_engagement = program_engagement.sort_values(by=['engagement'],ascending=False)
    
    group = {
    'top_programs':"|".join(program_engagement['programname'][0:min(2,len(contentsks))])
    , 'program_count':n_contentsks
    , 'respondents':n_personkeys
    , 'viewers_thousands':int(group_viewing.count()[0]/1000)
    , 'avg_engagement':round(group_viewing['engagement'].mean(),0)
    , 'median_engagement':round(group_viewing['engagement'].median(),0)
    , 'median_age':group_viewers['age'].median()
    , 'median_hh_income':group_viewers['householdincome'].median()
    , 'pct_children':pct_children 
    , 'pct_female':pct_female
    , 'pct_male':pct_male
    , 'all_programs':'|'.join(program_engagement['programname']) 
    , 'contentsks':program_engagement['contentsk'].tolist()
    , 'personkeys':personkeys
    }
    
    return(group)

In [17]:
# this function allows for recusivity so that larger communites can be broken down to smaller ones 
# to build a hierarchy
def build_community_groups(raw_communities, parent_group_id = "", group_level = 0):
    
    print("parent_group_id = {0}, len = {1}".format(parent_group_id, len(raw_communities)))
    
    community_groups = []
    i = 0
    
    for raw_community in raw_communities:
        print(i)
        community_group = build_community_group(raw_community)
        community_groups.append(community_group)
        i = i + 1

    community_groups = list(map(lambda x: build_community_group(x), raw_communities))
    
    df_community_groups = pd.DataFrame(community_groups, columns=['top_programs'
                                                ,'program_count'
                                                ,'respondents'
                                                ,'viewers_thousands'
                                                ,'avg_engagement'
                                                ,'median_engagement'
                                                ,'median_age'
                                                ,'median_hh_income'
                                                ,'pct_children'
                                                ,'pct_female'
                                                ,'pct_male'
                                                , 'all_programs'
                                                , 'contentsks'
                                                , 'personkeys'
                                                ])
    
    df_community_groups['group_id'] = [str(uuid.uuid4()).split('-')[0] for _ in range(len(df_community_groups.index))]
    
    df_community_groups['parent_group_id'] = parent_group_id
    
    df_community_groups['group_level'] = group_level
    
    # if the community has a certain amount of viewers and programs then continue to break it down
    x = df_community_groups[(df_community_groups['respondents']>20) & (df_community_groups['program_count']>10)]
    
    sub_community_groups = list()
    
    if x.shape[0] > 0:
      
        for i in range(0, x.shape[0] - 1):
            contentsks = x.loc[x.index[i], 'contentsks']
            personkeys = x.loc[x.index[i], 'personkeys']
            group_id = x.loc[x.index[i], 'group_id']
            
            subG = G.subgraph(contentsks + personkeys)  
            sub_communities = community.greedy_modularity_communities(subG)
            sub_community_groups.append(build_community_groups(sub_communities, parent_group_id = group_id, group_level = group_level + 1))  
            
        if len(sub_community_groups) > 0:
            df_community_groups = df_community_groups.append(pd.concat(sub_community_groups))

    
    return(df_community_groups)

In [18]:
# first let's create our iniital top-level viewer/content communities 
# this can take a while depending on your local machine's resources
initial_communities = community.greedy_modularity_communities(G)

In [19]:
# now let's recurse through the communities and break them down to build a detailed taxonomy
community_groups = build_community_groups(initial_communities)

parent_group_id = , len = 38
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
parent_group_id = 240a4bc5, len = 19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
parent_group_id = fc93699f, len = 7
0
1
2
3
4
5
6
parent_group_id = a04daf6a, len = 7
0
1
2
3
4
5
6
parent_group_id = 6d6c6007, len = 7
0
1
2
3
4
5
6
parent_group_id = 53d3be6f, len = 6
0
1
2
3
4
5
parent_group_id = 2e519cba, len = 8
0
1
2
3
4
5
6
7
parent_group_id = 8a008b21, len = 5
0
1
2
3
4
parent_group_id = b4e50837, len = 5
0
1
2
3
4
parent_group_id = 9004d403, len = 9
0
1
2
3
4
5
6
7
8
parent_group_id = c79163d8, len = 10
0
1
2
3
4
5
6
7
8
9
parent_group_id = 7d01dfd7, len = 10
0
1
2
3
4
5
6
7
8
9
parent_group_id = 08345542, len = 10
0
1
2
3
4
5
6
7
8
9
parent_group_id = 458f6c05, len = 8
0
1
2
3
4
5
6
7
parent_group_id = df7a5c12, len = 4
0
1
2
3
parent_group_id = 52903927, len = 5
0
1
2
3
4
parent_group_id = 9a5a43cf, len = 8
0
1
2
3
4
5
6
7
parent_group_id = 5012

In [20]:
# let's drop the smaller communities that have less than 6 viewers in them
community_groups = community_groups[(community_groups['respondents']>5)]

In [21]:
# take a peek at the table and look at the range of their respective attributes
community_groups.head()

,top_programs,program_count,respondents,viewers_thousands,avg_engagement,median_engagement,median_age,median_hh_income,pct_children,pct_female,pct_male,all_programs,contentsks,personkeys,group_id,parent_group_id,group_level
0,CURSE OF OAK ISLAND|GOLD RUSH,852,653,5,51.0,48.0,55.0,83.0,0.26,0.49,0.51,CURSE OF OAK ISLAND|GOLD RUSH|PAW PATROL|GOLD ...,"[P4128, P4880, P6071, P6339, P5902, P5685, P41...","[V852040-4, V24459-2, V6310591-2, V86290-2, V9...",240a4bc5,,0
1,THIS IS US|GREY'S ANATOMY,535,796,7,55.0,52.0,48.0,73.0,0.37,0.65,0.35,THIS IS US|GREY'S ANATOMY|911|EMPIRE|SURVIVOR|...,"[P4822, P5885, P5933, P4221, P4984, P4763, P57...","[V264091-2, V6474205-2, V6306280-2, V6042104-2...",7d01dfd7,,0
2,NCIS|BULL,305,760,9,57.0,53.0,63.0,88.0,0.20,0.56,0.44,"NCIS|BULL|BIG BANG THEORY, THE|BLUE BLOODS|NEW...","[P5333, P5430, P4597, P5940, P4635, P5462, P43...","[V6110180-1, V6431305-3, V712294-1, V6132469-2...",c44302e5,,0
3,GAME OF THRONES|LAST DANCE,197,318,0,53.0,50.0,48.0,83.0,0.41,0.48,0.52,GAME OF THRONES|LAST DANCE|SATURDAY NIGHT|WALK...,"[P6130, P4909, P4195, P6114, P6234, P5638, P62...","[V7772278-2, V6456016-1, V6241700-1, V139510-1...",96fcdfa1,,0
4,AMERICAN IDOL-SUN|SCHITTS CREEK,25,43,0,48.0,44.0,53.0,93.0,0.13,0.56,0.44,AMERICAN IDOL-SUN|SCHITTS CREEK|SPONGEBOB|PLL:...,"[P5240, P5003, P4772, P5220, P4527, P4700, P57...","[V7784966-2, V7768460-1, V296002-2, V6478152-2...",07a78856,,0


In [22]:
community_groups.describe()

,program_count,respondents,viewers_thousands,avg_engagement,median_engagement,median_age,median_hh_income,pct_children,pct_female,pct_male,group_level
count,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000
mean,30.566138,43.047619,0.158730,52.444444,49.793651,52.066138,83.740741,0.340106,0.550952,0.449048,2.174603
std,82.861408,102.510730,0.937511,5.044834,4.701150,10.999619,30.725604,0.243052,0.202928,0.202928,0.926094
min,1.000000,6.000000,0.000000,43.000000,40.000000,22.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,9.000000,0.000000,48.000000,46.000000,44.000000,63.000000,0.150000,0.430000,0.320000,2.000000
50%,9.000000,15.000000,0.000000,53.000000,50.000000,52.000000,83.000000,0.290000,0.560000,0.440000,2.000000
75%,21.000000,30.000000,0.000000,56.000000,53.000000,59.000000,98.000000,0.460000,0.680000,0.570000,3.000000
max,852.000000,796.000000,9.000000,66.000000,62.000000,99.000000,223.000000,1.000000,1.000000,1.000000,4.000000


Now we'll save the following datasets that we will reloade back into our database so that we can join with other data to do more analysis on our viewing taxonomy; or the output files can be used for analysis in other notebooks.

In [23]:
community_groups[['top_programs'
,'program_count'
,'respondents'
,'viewers_thousands'
,'avg_engagement'
,'median_engagement'
,'median_age'
,'median_hh_income'
,'pct_children'
,'pct_female'
,'pct_male'
,'personkeys'
,'contentsks'
,'group_id'
,'parent_group_id'
,'group_level'
,'all_programs']].to_csv("./output/viewing_communities.csv", index = False, index_label = False)


In [24]:
viewers.to_csv('./output/viewers.csv', index = False)

In [25]:
programs.to_csv('./output/programs.csv', index = False)